In [ ]:
# -*- coding: UTF-8 -*-
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import numpy as np

# 载入数据
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# 定义每个批次的大小
batch_size = 100
period_epoch = 31
acc_list = np.empty(period_epoch)
acc_list[0] = 0
lr_list = np.empty(period_epoch)

# 批次的大小
batch_size = 128
n_batch = mnist.train.num_examples // batch_size

x = tf.placeholder(tf.float32, [None,784])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
lr = tf.Variable(0.0008, dtype=tf.float32)

# 创建一个简单的神经网络
w1 = tf.Variable(tf.truncated_normal([784,500], stddev=0.1))
b1 = tf.Variable(tf.zeros([1,500]) )
L1 = tf.nn.tanh(tf.matmul(x,w1) + b1)
L1_drop = tf.nn.dropout(L1, keep_prob)

w2 = tf.Variable(tf.truncated_normal([500,300], stddev=0.1))
b2 = tf.Variable(tf.zeros([1,300]) )
L2 = tf.nn.tanh(tf.matmul(L1_drop,w2) + b2)
L2_drop = tf.nn.dropout(L2, keep_prob)

w3 = tf.Variable(tf.truncated_normal([300,10], stddev=0.1))
b3 = tf.Variable(tf.zeros([1,10]) )
prediction = tf.nn.softmax(tf.matmul(L2_drop,w3) + b3)

# 代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))

# 梯度下降法
# train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
# 不同的优化器
train_step = tf.train.AdamOptimizer(lr).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

# 得到一个布尔型列表，存放结果是否正确
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1)) #argmax 返回一维张量中最大值索引

# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) # 把布尔值转换为浮点型求平均数

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(period_epoch):
        
        for batch in range(n_batch):
            # 获得批次数据
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys,keep_prob:0.9})
        
        
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images,y:mnist.test.labels, keep_prob:0.9})
        acc_list[epoch] = acc
        if epoch >= 1:
            if acc_list[epoch-1] < acc_list[epoch]:
                sess.run(tf.assign(lr, 0.001 * (0.95**epoch)))
        learning_rate = sess.run(lr)
        lr_list[epoch] = learning_rate
        print("Iter " + str(epoch) + " Testing Accuracy: " + str(acc) + " Learning rate: " + str(learning_rate), end = '\r')
        
# 画图
fig = plt.figure()
axis = np.linspace(0, period_epoch, period_epoch)

ax1 = fig.add_subplot(211)  #2行1列第1幅图
ax1.plot(axis, acc_list, 'r-')
ax1.set_xlabel('epoch')
ax1.set_ylabel('accuracy')

ax2 = fig.add_subplot(212)  #2行1列第2幅图
ax2.plot(axis,lr_list, 'b-' )
ax2.set_xlabel('epoch')
ax2.set_ylabel('learning-rate')

plt.show()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
